## 1x1 Vectors Graph

A simple model with a single input layers and a single output layers, both vectors.

## Build and save the graph

Ensure you are working in a virtual environment with tensorflow >= 1.9 installed, and in the notebook's directory on the command line:

```bash
rm -r saved_graph
```

In [1]:
import tensorflow as tf

In [2]:
x = tf.placeholder(tf.float32, shape=(1,4), name="input_x")
w = tf.Variable([[1,0,1,0]], dtype=tf.float32, name="w")
z = tf.add(x, w, name="output_z" )

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    z_eval = sess.run([z], feed_dict={x: [[1,2,3,4]]})
    
    tf.saved_model.simple_save(sess, "./saved_graph", inputs={"input_x": x}, outputs={"output_z": z})
    
    print(z_eval) # [2,2,4,4]

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./saved_graph/saved_model.pb
[array([[2., 2., 4., 4.]], dtype=float32)]


## Freeze The Graph

Having used the `tf.saved_model.simple_save` function, now use the `freeze_graph` cli. Tensorflow also provides a python interface to the freeze_graph function.

On the command line:

```bash
mkdir frozen_graph

freeze_graph \
    --input_saved_model_dir=saved_graph \
    --saved_model_tags=serve \
    --output_graph=frozen_graph/frozen_graph.pb \
    --output_node_names=output_z
```

## Convert the Model to a TensorFlow Lite Format

Use the `toco` command line utility to convert the frozen graph to a tensorflow lite compatible file:

```bash
mkdir tflite

toco \
	--graph_def_file=frozen_graph/frozen_graph.pb \
	--output_file=tflite/model.tflite \
	--input_arrays=input_x \
	--input_shapes=1,4 \
	--output_arrays=output_z
```